In [1]:
%matplotlib widget

from experiments import Experiment, _DefaultFloatLogSlider, _DefaultIntSlider

from translocation_models import TranslocationModel, \
    SC2R, SC2R2Loops, DefectiveSC2R, \
    DiscSpiral, DefectiveDiscSpiral

from ipywidgets import Widget, FloatLogSlider, IntSlider, FloatRangeSlider, \
    HBox, VBox, HTML, Output, Layout
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.transforms import Affine2D
import pandas as pd
from pandas import DataFrame
import matplotlib as mpl


In [8]:
class VelocityVSBarrier(Experiment):
    # https://fr.wikipedia.org/wiki/%C3%89quation_d%27Eyring
    # https://fr.wikipedia.org/wiki/Loi_d%27Arrhenius
    def __init__(self):
        self._sc2r = SC2R()
        self._disc_spiral = DiscSpiral()
        super().__init__()

    def _construct_free_parameters(self) -> dict[str, Widget]:
        return {
            # Source for ATP/ADP ratio:
            # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6395684/#:~:text=The%20physiological%20nucleotide%20concentration%20ratio,is%20~10%E2%88%925).
            'barrier': FloatRangeSlider(
                value=[-10, 10], min=-10, max=10, continuous_update=False,
                description="ΔU/T:"),
            'atp_adp_ratio': _DefaultFloatLogSlider(
                value=100, min=0, max=4, readout_format='.1e',
                description="[ATP]/[ADP]:"),
            'equilibrium_atp_adp_ratio': _DefaultFloatLogSlider(
                value=1e-5, min=-7, max=-3, readout_format='.1e',
                description="([ATP]/[ADP])|eq.:"),
            'K_d_atp': _DefaultFloatLogSlider(
                value=0.1, description="K_d^ATP:"),
            'K_d_adp': _DefaultFloatLogSlider(description="K_d^ADP:"),
            'k_DT': _DefaultFloatLogSlider(description="k_DT:"),
            'k_h': _DefaultFloatLogSlider(description="k_h:"),
            'k_s': _DefaultFloatLogSlider(value=0.1, description="k_s:"),
            'k_up': _DefaultFloatLogSlider(description="k_↑:"),
            'n_protomers': _DefaultIntSlider(description="n_protomers:"),
            'k_extended_to_flat_up': _DefaultFloatLogSlider(description="k_⮫:"),
            'k_flat_to_extended_down': _DefaultFloatLogSlider(description="k_⮯:"),
            'k_flat_to_extended_up': _DefaultFloatLogSlider(description="k_⮭:"),
        }

    def _construct_constrained_parameters(self) -> dict[str, Widget]:
        return {
            'k_TD': HTML(description="k_TD:"),
            'k_down': HTML(description="k_↓:"),
            'k_h_bar': HTML(description="ꝁ_h:"),
            'k_flat_to_extended_down_bar': HTML(description="ꝁ_⮯:"),
            'k_extended_to_flat_down': HTML(description="k_⮩:"),
        }

    def _construct_gui(self) -> Widget:
        gui_plot = Output()
        gui_parameters = VBox([
            HTML(value="<h1>Velocity vs Potential</h1>"),

            HBox([self._free_parameters['barrier'],
                  HTML(value="Potential barrier over temperature range")]),

            HTML(value="<b>General Physical Parameters</b>"),
            HBox([self._free_parameters['atp_adp_ratio'],
                  HTML(value="ATP/ADP concentration ratio")]),
            HBox([self._free_parameters['equilibrium_atp_adp_ratio'],
                  HTML(value="Equilibrium ATP/ADP concentration ratio")]),
            HBox([self._free_parameters['K_d_atp'],
                  HTML(value="Protomer-ATP dissociation constant")]),
            HBox([self._free_parameters['K_d_adp'],
                  HTML(value="Protomer-ADP dissociation constant")]),
            HBox([self._free_parameters['k_DT'],
                  HTML(value="Effective ADP->ATP exchange rate")]),
            HBox([self._constrained_parameters['k_TD'],
                  HTML(value="Effective ATP->ADP exchange rate "\
                    "(constrained by Protomer-ATP/ADP exchange model)")]),
            HBox([self._free_parameters['k_h'],
                  HTML(value="ATP Hydrolysis rate")]),
            HBox([self._free_parameters['k_s'],
                  HTML(value="ATP Synthesis rate")]),

            HTML(value="<b>SC2R Model Physical Parameters</b>"),
            HBox([self._free_parameters['k_up'],
                  HTML(value="Translocation up rate")]),
            HBox([self._constrained_parameters['k_down'],
                  HTML(value="Translocation down rate "\
                       "(constrained by detailed balance)")]),

            HTML(value="<b>Disc-Spiral Model Physical Parameters</b>"),
            HBox([self._free_parameters['n_protomers'],
                  HTML(value="Number of protomers")]),
            HBox([self._constrained_parameters['k_h_bar'],
                  HTML(value="Effective ATP hydrolysis rate")]),
            HBox([self._free_parameters['k_extended_to_flat_up'],
                  HTML(value="Spiral->disc up translocation rate")]),
            HBox([self._free_parameters['k_flat_to_extended_down'],
                  HTML(value="Disc->spiral down translocation rate")]),
            HBox([self._constrained_parameters['k_flat_to_extended_down_bar'],
                  HTML(value="Effective disc->spiral down translocation rate")]),
            HBox([self._free_parameters['k_flat_to_extended_up'],
                  HTML(value="Disc->spiral up translocation rate")]),
            HBox([self._constrained_parameters['k_extended_to_flat_down'],
                  HTML(value="Spiral->disc down rate "\
                       "(constrained by detailed balance)")]),
        ])

        gui = HBox([gui_plot, gui_parameters],
                   layout=Layout(align_items='center'))

        return gui
    
    def _run(self) -> None:
        models = [self._sc2r, self._disc_spiral]
        # Update models<->GUI
        self._update_models_free_parameters(models, self._free_parameters)
        self._update_gui_constrained_parameters(models,
                                                self._constrained_parameters)
        
        # Add potentials in range
        min, max = self._free_parameters['barrier'].value
        print(min, max)
        barriers = np.linspace(min, max, 100)
        velocities = {model: [] for model in models}
        for barrier in barriers:
            for model in models:
                for _, _, attributes in model.kinetic_scheme.edges(data=True):
                    # For each displacement edge, we multiply the rate with the 
                    # barrier Boltzmann factor
                    if 'position' in attributes and attributes['position'] > 0:
                        old_rate = attributes['rate']
                        attributes['rate'] = (
                            lambda old_rate=old_rate: 
                                old_rate() * np.exp(-barrier))
                    elif 'position' in attributes and attributes['position'] < 0:
                        old_rate = attributes['rate']
                        attributes['rate'] = (
                            lambda old_rate=old_rate: 
                                old_rate() * np.exp(barrier))
                print(model.average_velocity()) # TODO debug why NaN -> understand why negative velocity very quickly
                velocities[model].append(model.average_velocity())

        gui_plot = self._gui.children[0]
        with gui_plot:
            gui_plot.clear_output(wait=True)
            plt.close('VelocityVSBarrier')
            fig = plt.figure('VelocityVSBarrier')
            fig.canvas.header_visible = False
            fig.canvas.footer_visible = False
            fig.canvas.toolbar_visible = False
            ax = fig.add_subplot(111)

            # Plot velocities vs [ATP]/[ADP]
            ax.plot(barriers,
                    velocities[self._sc2r],
                    label='SC/2R',
                    color='#DDAA33')
            ax.plot(barriers,
                    velocities[self._disc_spiral],
                    label='Disc-Spiral',
                    color='#004488')

            ax.set_xlabel("ΔU/T")
            ax.set_ylabel("❬v❭ [Residue ∙ k]")
            ax.legend()
            plt.show()





In [9]:
velocity_vs_barrier = VelocityVSBarrier()

-10.0 10.0
0.9950001316393455
3.1380253463434022
0.995024873939185
3.138075310409543
0.995024875621719
3.138075313807185
0.9950248756218907
3.1380753138075312
0.9950248756218907
3.1380753138075312
0.9950248756218908
3.1380753138075317
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.138075313807531
0.9950248756218906
3.1380753138075317
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.138075313807532
0.9950248756218907
3.1380753138075317
0.9950248756218908
3.1380753138075312
0.9950248756218906
3.1380753138075312
0.9950248756218907
3.138075313807532
0.9950248756218907
3.138075313807532
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.1380753138075312
0.9950248756218907
3.1380753138075312
0.9950248756218907
3.1380753138075317
0.9950248756218907
3.1380753138075312
0.9950248

/tmp/ipykernel_16088/3604922973.py:127: RuntimeWarning: overflow encountered in scalar multiply
  old_rate() * np.exp(barrier))
